In [24]:
import gzip
import json
import glob

import shutil
import pandas as pd
import numpy as np
import datetime
from collections import defaultdict

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


## Data Loading

In [25]:
types = r"data/*.gz"
files = glob.glob(types)
files

['data/receipts.json.gz', 'data/users.json.gz', 'data/brands.json.gz']

In [26]:

receipts = pd.read_json('data/receipts.json', lines=True)
brands = pd.read_json('data/brands.json', lines=True)

with gzip.open('data/users.json.gz', 'rb') as file_in:
    with open('data/users.json.gz'.replace('.gz',''), 'wb') as file_out:
        shutil.copyfileobj(file_in, file_out)

users = pd.read_json('data/users.json', lines=True)

## Data Prepocessing

### Receipts

In [27]:
receipts['_id'] = pd.json_normalize(receipts['_id'])

### Date ###
receipts['createDate'] = pd.json_normalize(receipts['createDate'])
receipts['dateScanned'] = pd.json_normalize(receipts['dateScanned'])

receipts['finishedDate'] = receipts['finishedDate'].map(lambda x: {'$date': 0} if pd.isna(x) else x)
receipts['finishedDate'] = pd.json_normalize(receipts['finishedDate'])

receipts['modifyDate'] = receipts['modifyDate'].map(lambda x: {'$date': 0} if pd.isna(x) else x)
receipts['modifyDate'] = pd.json_normalize(receipts['modifyDate'])

receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].map(lambda x: {'$date': 0} if pd.isna(x) else x)
receipts['pointsAwardedDate'] = pd.json_normalize(receipts['pointsAwardedDate'])

receipts['purchaseDate'] = receipts['purchaseDate'].map(lambda x: {'$date': 0} if pd.isna(x) else x)
receipts['purchaseDate'] = pd.json_normalize(receipts['purchaseDate'])

### Convert Timestamp ###
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']

for col in date_columns:
    receipts[col] = receipts[col].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S') if x != 0 else None)


receipts.head(5)


,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:36,2021-01-03 07:25:31,500.0,2021-01-02 16:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:48,2021-01-03 07:24:43,150.0,2021-01-02 07:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 07:25:37,2021-01-03 07:25:37,None,2021-01-03 07:25:42,None,5.0,2021-01-02 16:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:39,2021-01-03 07:25:34,5.0,2021-01-02 16:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 07:25:06,2021-01-03 07:25:06,2021-01-03 07:25:11,2021-01-03 07:25:11,2021-01-03 07:25:06,5.0,2021-01-02 07:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


### Rewards Receipt Item List

In [28]:
### rewardsReceiptItemList ###

temp = receipts[receipts['rewardsReceiptItemList'].notna()]
rewardsReceiptItemList = temp[['_id','rewardsReceiptItemList']]
receipts.drop('rewardsReceiptItemList',axis=1, inplace=True) ### drop rewardsReceiptItemList column from receipts dataframe

itemlist = defaultdict(list)

for products in rewardsReceiptItemList.values: 
    length = len(products[1])
    
    for j in range(length):
        itemlist['_id'].append(products[0])
        itemlist['products'].append(products[1][j])

itemlist = pd.DataFrame(itemlist)
itemlist = pd.concat([itemlist['_id'], pd.json_normalize(itemlist['products'])], axis = 1)


In [29]:
itemlist = itemlist[['_id', 'barcode', 'description','brandCode']]
itemlist.head(5)

,_id,barcode,description,brandCode
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,NaN


### Brands

In [30]:
brands['_id'] = pd.json_normalize(brands['_id'])

In [31]:
cpg = pd.json_normalize(brands['cpg'])
cpg.columns = ['ref', "oid"]
brands = pd.concat([brands,cpg], axis=1)
brands.drop('cpg', axis=1, inplace=True)

In [32]:
brands.head()

,_id,barcode,category,categoryCode,name,topBrand,brandCode,ref,oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


### Users

In [33]:
users['_id'] = pd.json_normalize(users['_id'])
users['createdDate'] = pd.json_normalize(users['createdDate'])
users['lastLogin'] = users['lastLogin'].map(lambda x: {'$date': 0} if pd.isna(x) else x)
users['lastLogin'] = pd.json_normalize(users['lastLogin'])

### Convert Timestamp ###
date_columns = ['createdDate', 'lastLogin']
for col in date_columns:
    users[col] = users[col].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000).strftime('%Y-%m-%d %H:%M:%S'))


In [34]:
users.head(5)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 07:25:30,2021-01-03 07:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI


## Second: Write a query that directly answers a predetermined question from a business stakeholder

In [35]:
import sqlite3

with sqlite3.connect(":memory:") as conn:
    receipts.to_sql(name="receipts", con=conn, index=False)
    itemlist.to_sql(name="itemlist", con=conn, index=False)
    users.to_sql(name="users", con=conn, index=False)
    brands.to_sql(name="brands", con=conn, index=False)

### a) What are the top 5 brands by receipts scanned for most recent month?


In [36]:
query = """

SELECT 
        itemlist.description,
        COUNT(*) AS "Number of Brands"
FROM itemlist
WHERE itemlist._id IN 
                    (                    
                    SELECT DISTINCT _id AS "receipts_id"
                    FROM receipts
                    WHERE strftime("%m", dateScanned) =  (SELECT STRFTIME("%m", MAX(dateScanned))
                                                            FROM receipts )
                        AND strftime("%Y", dateScanned) = (SELECT STRFTIME("%Y",MAX(dateScanned))
                                                            FROM receipts)                                                                
                    )
    AND itemlist.description IS NOT NULL
GROUP BY itemlist.description
ORDER BY 2  
LIMIT 5

    
"""
results = pd.read_sql(query, con=conn)
results

,description,Number of Brands
0,mueller austria hypergrind precision electric ...,10
1,thindust summer face mask - sun protection nec...,10


### b) How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [37]:
query = """

SELECT 
        IFNULL(itemlist.brandCode,'NULL'),
        COUNT(*) AS 'Number of Brands'
FROM itemlist
WHERE itemlist._id IN 
                    (                    
                    SELECT DISTINCT _id AS "receipts_id"
                            
                    FROM receipts
                    WHERE strftime("%m", dateScanned) =  (SELECT STRFTIME("%m", DATE(MAX(dateScanned),'-1 months'))
                                                            FROM receipts )
                        AND strftime("%Y", dateScanned) = (SELECT STRFTIME("%Y",DATE(MAX(dateScanned),'-1 months'))
                                                            FROM receipts)                                                                
                    )
    
GROUP BY itemlist.brandCode
ORDER BY 2 DESC    
LIMIT 5       
    
"""
results = pd.read_sql(query, con=conn)
results

,"IFNULL(itemlist.brandCode,'NULL')",Number of Brands
0,NULL,190
1,BRAND,3
2,MISSION,2
3,VIVA,1


In [38]:
query = """

SELECT 
        IFNULL(itemlist.brandCode,'NULL'),
        COUNT(*) AS 'Number of Brands'
FROM itemlist
WHERE itemlist._id IN 
                    (                    
                    SELECT DISTINCT _id AS "receipts_id"
                            
                    FROM receipts
                    WHERE strftime("%m", dateScanned) =  (SELECT STRFTIME("%m", DATE(MAX(dateScanned),'-2 months'))
                                                            FROM receipts )
                        AND strftime("%Y", dateScanned) = (SELECT STRFTIME("%Y",DATE(MAX(dateScanned),'-2 months'))
                                                            FROM receipts)                                                                 
                    )
                    
GROUP BY itemlist.brandCode
ORDER BY 2 DESC    
LIMIT 5       
    
"""
results = pd.read_sql(query, con=conn)
results

,"IFNULL(itemlist.brandCode,'NULL')",Number of Brands
0,NULL,4061
1,HY-VEE,291
2,BEN AND JERRYS,180
3,PEPSI,93
4,KROGER,89


### c) When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?


In [39]:
query = """

SELECT 
    receipts.rewardsReceiptStatus,
    AVG(receipts.totalSpent) AS 'Average Spent'
    
FROM receipts
WHERE receipts.totalSpent IS NOT NULL
GROUP BY receipts.rewardsReceiptStatus

"""
results = pd.read_sql(query, con=conn)
results

,rewardsReceiptStatus,Average Spent
0,FINISHED,80.854305
1,FLAGGED,180.451739
2,PENDING,28.032449
3,REJECTED,23.326056


### d) When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?


In [40]:
query = """

SELECT 
    receipts.rewardsReceiptStatus,
    SUM(receipts.purchasedItemCount) AS 'Total Items Count'
    
FROM receipts
WHERE receipts.purchasedItemCount IS NOT NULL
GROUP BY receipts.rewardsReceiptStatus

"""
results = pd.read_sql(query, con=conn)
results


,rewardsReceiptStatus,Total Items Count
0,FINISHED,8184.0
1,FLAGGED,1014.0
2,REJECTED,173.0


### e) Which brand has the most spend among users who were created within the past 6 months?


In [41]:
query = """

WITH users_6months_id AS (
SELECT DISTINCT _id    
FROM users
WHERE createdDate >= ( SELECT DATE(MAX(createdDate),'start of month','-6 months') 
                      FROM users )
)

SELECT 
    itemlist.brandCode,
    SUM(receipts.totalSpent) AS 'totalspent'
    
FROM receipts
INNER JOIN users_6months_id
    ON receipts.userId = users_6months_id._id
INNER JOIN itemlist
    ON receipts._id = itemlist._id 
WHERE itemlist.brandCode IS NOT NULL
GROUP BY itemlist.brandCode
ORDER BY 2 DESC
LIMIT 1 

"""

results = pd.read_sql(query, con=conn)
results


,brandCode,totalspent
0,BEN AND JERRYS,197337.68


### f) Which brand has the most transactions among users who were created within the past 6 months?


In [42]:
query = """

WITH users_6months_id AS (
SELECT DISTINCT _id    
FROM users
WHERE createdDate >= ( SELECT DATE(MAX(createdDate),'start of month','-6 months') 
                      FROM users )
)

SELECT 
    itemlist.brandCode,
    COUNT(*) AS 'Transactions Count'
    
FROM receipts
INNER JOIN users_6months_id
    ON receipts.userId = users_6months_id._id
INNER JOIN itemlist
    ON receipts._id = itemlist._id 
WHERE itemlist.brandCode IS NOT NULL
GROUP BY itemlist.brandCode
ORDER BY 2 DESC
LIMIT 1 

"""

results = pd.read_sql(query, con=conn)
results


,brandCode,Transactions Count
0,HY-VEE,291


## Third: Evaluate Data Quality Issues in the Data Provided


In [85]:
print(f"Number of total Brands from Brands Table : {brands['name'].nunique()}")

brands['barcode'] = brands['barcode'].astype(str)
real_brands = brands[~brands['name'].str.contains(r'^(?=.*test)(?=.*test brand)')]
print(f"Number of Brands that is not a test brand : {real_brands['name'].nunique()}")

receipts_barcode = itemlist[itemlist['barcode'].isin(real_brands['barcode'].values)]
print(f"Number of barcodes from brands table that matches barcodes in the receipts table : {receipts_barcode['barcode'].nunique()}")

item_brandcode = itemlist[itemlist.brandCode.notna()]
matching_brandcode = item_brandcode[item_brandcode['brandCode'].isin(brands['brandCode'].unique())]['brandCode'].nunique()

print(f"Number of brandcodes from brands table that matches brandcodes in receipts table : {matching_brandcode}")


Number of total Brands from Brands Table : 1156
Number of Brands that is not a test brand : 728
Number of barcodes from brands table that matches barcodes in the receipts table : 16
Number of brandcodes from brands table that matches brandcodes in receipts table : 41


One crucial data quality issue I discovered is that a large number of test brands and null values exist in the brands table. This made it difficult to find the names of the brands purchased in the receipts table in order to match the names of brands. As we can see from the above results, only 728 brands were left when test brands were excluded from the brands' table. Furthermore, the itemlist table which was acquired from the receipts table did not contain a uuid of items, which made it hard to find the brand of the items. In order to solve this issue, I used barcodes of those 728 brands to see the types of brands purchased by users in the receipts. When brandcodes were compared, it was slightly better with 41 matching brands but it still shows poor matching rate between receipts and brands data. However, I could only match 16 of them in the receipts table. Eventually, I had to use a description of the items to find the brand.

## Fourth: Communicate with Stakeholders

Hi,

As I was reviewing different sources of data for analytic purposes, it came to my attention that there are a very small number of brand types that could be used to find business insights.

My objective was to find the top brands purchased by users. There are two sources of data tables which contain tables of brands. One is from users' purchased receipts data table and another is from a data table which only contains brand information. 

However, there were too many test values in a brand table and I had to look at the receipts data table to find valid brand names. However, the receipts table also had an issue with a large number of NULL values which indicates empty brand name values. Therefore, it was hard to see what brands were most frequently or least purchased from users' receipts data.  

In order to fully understand and resolve this issue, I would like to know the method implemented by the company to obtain the data. From my experience, there are usually two ways to acquire this kind of data. One is a scraping method which refers to the process of using bots to extract data, while the API method provides direct access to the data. My assumption is that we are currently using scraping methods as there are way too many empty values in the data, but I would like to make sure that I am on the right page with the method used. If I am correct, we will have to improve our scraping skills to upgrade the quality of our data. Another possible suggestion is to manually create every possible brand and items of each brand to improve the quality of the data item. By doing so, it will be easier to identify top brands and items purchased by analyzing the receipts. I would really appreciate your clearance on this matter and hope this could result in improvement of the data quality.

Best,
<br>
Edward Her